In [50]:
from selenium import webdriver
from selenium.webdriver.common.by import By 
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC 
from bs4 import BeautifulSoup
import time 
import re
import json 
import pandas as pd
from datetime import datetime

## id 와 password 입력

In [51]:
user_id=""
user_passwd=""

In [52]:
login_option="instagram" # facebook or instagram 
driver_path="chromedriver.exe" 
instagram_id_name="username" 
instagram_pw_name="password"
instagram_login_btn=".sqdOP.L3NKy.y3zKF "
instagram_information_save_name = ".sqdOP.yWX7d.y3zKF"
instagram_alarm_name = ".aOOlW.HoLwm"
is_information_save = False
is_alarm_setting = False

driver = webdriver.Chrome("./chromedriver")

In [53]:
print(f"login start - option {login_option}")
login_url = "https://www.instagram.com/accounts/login/"
driver.get(login_url)
print(login_option)

login start - option instagram
instagram


In [54]:
if login_option == "instagram": 
    time.sleep(2)
    try: 
        instagram_id_form = driver.find_element_by_name(instagram_id_name) 
        instagram_id_form.send_keys(user_id) 
        
        instagram_pw_form = driver.find_element_by_name(instagram_pw_name) 
        instagram_pw_form.send_keys(user_passwd) 
        
        login_ok_button = driver.find_element_by_css_selector(instagram_login_btn) 
        login_ok_button.click() 
        is_login_success = True
        
        time.sleep(3)
        if is_information_save == False and is_alarm_setting == False:
            information_save_btn = driver.find_element_by_css_selector(instagram_information_save_name)
            information_save_btn.click()
            is_information_save = True
            
            alarm_setting_btn = driver.find_element_by_css_selector(instagram_alarm_name)
            alarm_setting_btn.click()
            is_alarm_setting = True
            
    except: 
        print("instagram login fail") 
        is_login_success = False

## 위 내용까지 로그인

In [55]:
my_account_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/span"
my_account_button = driver.find_element_by_xpath(my_account_xpath)
my_account_button.click()

my_profile_full_xpath = "/html/body/div[1]/section/nav/div[2]/div/div/div[3]/div/div[6]/div[2]/div[2]/div[2]/a[1]/div"
my_profile_button = driver.find_element_by_xpath(my_profile_full_xpath)
my_profile_button.click()

In [56]:
time.sleep(1.5)
# 게시물 개수
post_count = driver.find_element_by_xpath("/html/body/div[1]/section/main/div/header/section/ul/li[1]/div/span").text
post_count = int(post_count)

print("게시물 수 : ",post_count)

게시물 수 :  13


### 현재 모든 댓글보기 클릭 가능 (while문 이용)
<li> 1. 모든 답글이 보여진 상태에서 html parser 이용
<li> 2. css 또는 xpath 이용

In [73]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

for i in range(post_count-1) :
    # 날짜 데이터 크롤링
    time.sleep(1.5)
    #date = driver.find_element_by_css_selector("div.k_Q0X.I0_K8.NnvRN > a > time").text
    date = driver.find_element_by_css_selector('time.FH9sR.Nzb55')
    date_info = pd.to_datetime(date.get_attribute('datetime')).normalize()
    date = str(date_info.year) + "년 " + str(date_info.month) + "월 " + str(date_info.day) +"일"
    print("date : ", date)

    # 장소 크롤링
    time.sleep(0.5)
    try : 
        place = driver.find_element_by_css_selector("body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.UE9AK > div > header > div.o-MQd.z8cbW > div.M30cS > div > div.JF9hh > div > a").text
    except : 
        place = None
    print("place : ",place)
    
    # 게시글 내용 크롤링
    #body > div.RnEpo._Yhr4 > div.pbNvD.QZZGH.bW6vo > div > article > div > div.HP0qD > div > div > div.eo2As > div.EtaWk > ul
    try : 
        post_content = driver.find_element_by_xpath("/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/div/li/div/div/div[2]/div[1]/span").text    
    except :
        post_content = None
    # 정제 필요
    print("original post_content : ", post_content)
    
    # 본문 속 해시태그 정제
    if post_content != None :
        post_content_tag= []
        post_content = re.findall("#[A-Za-z0-9가-힣]+", post_content)
        content = ''.join(post_content).replace("#"," ") # "# 제거"
        content_split = content.split()
        
        for tag in content_split :
            post_content_tag.append(tag)
    print("tag in post_content : ", post_content_tag)
    
    # 댓글 크롤링
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    comment_count = 0

    # 댓글 있는지 확인
    try  :
        ul_list = soup.find_all('ul',  'Mr508')
        comment_count = len(ul_list)
    
        for d in ul_list : 
            user_id = d.find("h3","_6lAjh").find("a").get_text()
            print("user id : ",user_id)
        
            comment = d.find("div" , "C4VMK").find_all("span")[1].get_text()
            print("comment : ",comment)
    except :
        # 댓글 없으면
        comment_count = 0
    else : 
        # 댓글 개수 적용하여 대댓글 열기버튼 xpath 추출
        print("comment_count : ",comment_count)
        for i in range(1,comment_count+1) :
            button_xpath = "/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[" + str(i) + "]/li/ul/li/div/button"
            try : 
                # 대댓글 열기버튼 있으면 클릭
                driver.find_element_by_xpath(button_xpath).click()
                button_check = driver.find_element_by_class_name("EizgU").text
                print(button_check)
                while "답글 숨기기" not in button_check : 
                    driver.find_element_by_xpath(button_xpath).click()
                    button_check = driver.find_element_by_class_name("EizgU").text
                    
            except :
                button_xpath = None
            
        # 대댓글이 모두 열려있는 상태에서 html parser        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        ul_list = soup.find_all('ul',  'Mr508') 
        
        for d in ul_list :
            try : 
                print("\n대댓글이 존재합니다 !! \n")
                comment_comment_id = d.find("ul","TCSYW").find("h3","_6lAjh").find("a").get_text()
                #print("comment_comment id:",comment_comment_id)
                comment_comment = d.find("ul","TCSYW").find_all("span")[3].get_text()
                #print("comment_comment : ", comment_comment)
            except:
                comment_comment_id = None
                comment_comment = None
            print("comment_comment id:",comment_comment_id)
            print("comment_comment : ", comment_comment)
            
    # 다음 버튼
    next_post_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.Z2Inc._7c9RR > div > div.l8mY4.feth3 > button")
    next_post_button.click()
    time.sleep(0.5)
    
# 닫기 버튼    
exit_button = driver.find_element_by_css_selector("div.RnEpo._Yhr4 > div.NOTWr > button > div > svg")
exit_button.click()

date :  2022년 2월 15일
place :  None
original post_content :  #문자메세지
tag in post_content :  ['문자메세지']
user id :  ___kiss_o
comment :  이거 테스트 ㄱ
user id :  ___kiss_o
comment :  #나는지금기쁘다
user id :  jo_o_han
comment :  ?ㅡ?
user id :  perarduaadastra__
comment :  이게뭐에요?
comment_count :  4
답글 보기(9개)
답글 숨기기
답글 숨기기

대댓글이 존재합니다 !! 

comment_comment id: None
comment_comment :  None

대댓글이 존재합니다 !! 

comment_comment id: go_ne_jin
comment_comment :  @___kiss_o #나는지금아무생각이없다

대댓글이 존재합니다 !! 

comment_comment id: None
comment_comment :  None

대댓글이 존재합니다 !! 

comment_comment id: None
comment_comment :  None
date :  2021년 8월 29일
place :  낙산사
original post_content :  개강 전 finale
.
.
.
.
.
.
.
.
.
.
혜진 귤 왜 안까 ?
tag in post_content :  []
user id :  suuout
comment :  👶
user id :  jo_o_han
comment :  👀
user id :  xiyehn
comment :  혜진 ~ 귤 까조 🍊
comment_count :  3
답글 보기(1개)
답글 숨기기

대댓글이 존재합니다 !! 

comment_comment id: go_ne_jin
comment_comment :  @sweeni_54 뻬이뻬 K대면에서 만나…

대댓글이 존재합니다 !! 

comment_comment id: None
co

## 아래는 첫번째 게시글로만 test하는 코드

In [70]:
# 첫 게시물
first_post_css = "a > div.eLAPa > div._9AhH0"
first_post_button = driver.find_element_by_css_selector(first_post_css)
first_post_button.click()

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
comment_count = 0

# 댓글 있는지 확인
try  :
    ul_list = soup.find_all('ul',  'Mr508')
    comment_count = len(ul_list)
    
    for d in ul_list : 
        user_id = d.find("h3","_6lAjh").find("a").get_text()
        print("user id : ",user_id)
        
        comment = d.find("div" , "C4VMK").find_all("span")[1].get_text()
        print("comment : ",comment)
except :
    # 댓글 없으면
    comment_count = 0
else : 
    # 댓글 개수 적용하여 대댓글 열기버튼 xpath 추출
    print("comment_count : ",comment_count)
    for i in range(1,comment_count+1) :
        button_xpath = "/html/body/div[6]/div[3]/div/article/div/div[2]/div/div/div[2]/div[1]/ul/ul[" + str(i) + "]/li/ul/li/div/button"
        try : 
            # 대댓글 열기버튼 있으면 클릭
            driver.find_element_by_xpath(button_xpath).click()
            button_check = driver.find_element_by_class_name("EizgU").text
            print(button_check)
            while "답글 숨기기" not in button_check : 
                driver.find_element_by_xpath(button_xpath).click()
                button_check = driver.find_element_by_class_name("EizgU").text
                    
        except :
            button_xpath = None

        ids  = driver.find_elements_by_css_selector('div.C4VMK > h2 > div > span > a')
        replies = driver.find_elements_by_css_selector('div.C4VMK > div.MOdsX > span')
        
        for i in range(len(ids)) :
            print(ids[i])
        for i in range(len(replies)) :
            print(replies[i])
    
"""
    # 추가 댓글이 모두 열려있는 상태에서 html parser        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    ul_list = soup.find_all('ul',  'Mr508')
    print(ul_list)
    
    for d in ul_list :
        try :                   
            comment_comment = d.find("ul","TCSYW").find_all("span")[3].get_text()
            comment_comment_id = d.find("ul","TCSYW").find("h3","_6lAjh").find("a").get_text()
            print("comment_comment id:",comment_comment_id)
            print("comment_comment : ", comment_comment)
        except:
            comment_comment_id = None
            comment_comment = None

print("끝")"""
            
            

user id :  ___kiss_o
comment :  이거 테스트 ㄱ
user id :  ___kiss_o
comment :  #나는지금기쁘다
user id :  jo_o_han
comment :  ?ㅡ?
user id :  perarduaadastra__
comment :  이게뭐에요?
comment_count :  4
<selenium.webdriver.remote.webelement.WebElement (session="dd502676707bca93334692928c2c22a6", element="8c0eaf23-b4a7-4923-a906-04902e2bd103")>
답글 보기(9개)
<selenium.webdriver.remote.webelement.WebElement (session="dd502676707bca93334692928c2c22a6", element="8c0eaf23-b4a7-4923-a906-04902e2bd103")>
답글 숨기기
<selenium.webdriver.remote.webelement.WebElement (session="dd502676707bca93334692928c2c22a6", element="8c0eaf23-b4a7-4923-a906-04902e2bd103")>
답글 숨기기
<selenium.webdriver.remote.webelement.WebElement (session="dd502676707bca93334692928c2c22a6", element="8c0eaf23-b4a7-4923-a906-04902e2bd103")>


'\n    # 추가 댓글이 모두 열려있는 상태에서 html parser        \n    html = driver.page_source\n    soup = BeautifulSoup(html, \'html.parser\')\n    ul_list = soup.find_all(\'ul\',  \'Mr508\')\n    print(ul_list)\n    \n    for d in ul_list :\n        try :                   \n            comment_comment = d.find("ul","TCSYW").find_all("span")[3].get_text()\n            comment_comment_id = d.find("ul","TCSYW").find("h3","_6lAjh").find("a").get_text()\n            print("comment_comment id:",comment_comment_id)\n            print("comment_comment : ", comment_comment)\n        except:\n            comment_comment_id = None\n            comment_comment = None\n\nprint("끝")'